In [23]:
import pandas as pd
import numpy as np
df = pd.read_csv(
    "../datasets/raw/agmarknet_punjab_tomato_raw.csv",
    dtype={'Arrivals (Quintals)': 'float'},
    parse_dates=['Reported Date'],
    low_memory=False
)


In [24]:
rows_to_drop = df[df['State Name'] == '-'].index
df.drop(rows_to_drop, inplace=True)
df.drop(columns=['State Name', 'District Name', 'Market Name', 'Variety', 'Group'], inplace=True)

In [25]:
df["Min Price (Rs./Quintal)"] = df["Min Price (Rs./Quintal)"].str.replace(",", "").astype(float)
df["Max Price (Rs./Quintal)"] = df["Max Price (Rs./Quintal)"].str.replace(",", "").astype(float)
df["Modal Price (Rs./Quintal)"] = df["Modal Price (Rs./Quintal)"].str.replace(",", "").astype(float)
df["Reported Date"] = pd.to_datetime(df["Reported Date"], format="%Y-%m-%d")
df["Arrivals (Tonnes)"] = df["Arrivals (Tonnes)"].str.replace(",", "").astype(float)
df["Arrivals (Tonnes)"] = df["Arrivals (Tonnes)"]*10
df.rename(columns={"Arrivals (Tonnes)": "Arrivals (Quintals)"}, inplace=True)

In [26]:
df.rename(columns={
    "Min Price (Rs./Quintal)": "min_price",
    "Max Price (Rs./Quintal)": "max_price",
    "Modal Price (Rs./Quintal)": "modal_price",
    "Arrivals (Quintals)": "arrivals",
    "Reported Date": "date"
}, inplace=True)

In [27]:
df['weighted_modal'] = df['arrivals'] * df['modal_price']
df['price_range'] = df['max_price'] - df['min_price']

In [28]:
daily = df.groupby('date').agg({
    'arrivals': 'sum',
    'min_price': 'min',
    'max_price': 'max',
    'weighted_modal': 'sum',
    'price_range': 'mean'
}).reset_index()

daily['modal_price'] = daily['weighted_modal'] / daily['arrivals']
daily = daily.drop(columns=['weighted_modal'])

In [ ]:
# # for tomato dataset
# missing_row = pd.DataFrame({
#     'date': [pd.to_datetime('2023-08-06')],
#     'arrivals': [np.nan], 
#     'min_price': [np.nan],
#     'max_price': [np.nan],
#     'price_range': [np.nan],
#     'modal_price': [np.nan]
# })

# daily = pd.concat([daily, missing_row], ignore_index=True)
# daily = daily.sort_values('date').reset_index(drop=True)
# daily[['arrivals', 'min_price', 'max_price', 'price_range', 'modal_price']] = daily[
#     ['arrivals', 'min_price', 'max_price', 'price_range', 'modal_price']
# ].ffill()

In [30]:
daily['year'] = daily['date'].dt.year
daily['month'] = daily['date'].dt.month
daily['day_of_week'] = daily['date'].dt.dayofweek

In [31]:
daily['lagged_price_7'] = daily['modal_price'].shift(7)
daily['lagged_price_30'] = daily['modal_price'].shift(30)
daily['rolling_mean_7'] = daily['modal_price'].shift(1).rolling(7).mean()
daily['rolling_mean_30'] = daily['modal_price'].shift(1).rolling(30).mean()
daily['exp_smooth'] = daily['modal_price'].shift(1).ewm(span=7, adjust=False).mean()
daily['price_change'] = daily['modal_price'].pct_change(fill_method=None).shift(1) * 100
daily['log_modal'] = np.log1p(daily['modal_price'])
mean_7 = daily['modal_price'].shift(1).rolling(7).mean()
std_7 = daily['modal_price'].shift(1).rolling(7).std()
daily['z_score_7_rolling'] = (daily['modal_price'] - mean_7) / std_7

In [32]:
daily = daily.dropna().reset_index(drop=True)
daily = daily[daily['date'] >= pd.to_datetime('2020-01-01')]

In [33]:
daily.to_csv("../datasets/processed/agmarknet_punjab_tomato.csv", index=False)